In [5]:
from tqdm import tqdm

In [ ]:
# read in a text file
with open('nietzsche.txt', 'r') as f:
    text = f.read()

print(f"length of text: {len(text)} characters")

length of text: 600901 characters
